# Train on separate images (CNN-Only) on InceptionV3 output

In [1]:
import h5py
import numpy as np
import os
import random
from os.path import join
from keras import layers
from keras.models import Model
from keras.utils import to_categorical

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dataset = 'UCF11'
nb_classes = int(dataset[3:])

In [3]:
def train_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        images = []
        labels = []

        for _ in range(batch_size):
            
            random_filename = random.choice(all_files)
            
            with h5py.File(join(dataset_dir, random_filename), 'r') as hf:
                frames = hf['inception'][:]
                fr_labels = hf['Y'][:]
                random_idx = np.random.randint(frames.shape[0])
                
                images.append(frames[random_idx])
                labels.append(fr_labels)
                
        images = np.array(images)
        labels = to_categorical(np.array(labels), nb_classes)
        
        yield images, labels
        
        

def valid_generator(dataset_dir, batch_size):
    
    all_files = os.listdir(dataset_dir)
    
    while True:
        
        for filename in all_files:

            with h5py.File(join(dataset_dir, filename), 'r') as hf:
                frames = hf['inception'][:]
                single_label = hf['Y'][:][0]

                fr_labels = np.array([single_label] * frames.shape[0])

                yield frames, to_categorical(fr_labels, nb_classes)

In [4]:
separate_dataset_dir = join('datasets', dataset, 'separate_frames_50_h_120_w_160')
train_dir = join(separate_dataset_dir, 'train')
valid_dir = join(separate_dataset_dir, 'valid')

train_samples_count = len(os.listdir(train_dir))
valid_samples_count = len(os.listdir(valid_dir))

with h5py.File(join(train_dir, os.listdir(train_dir)[0])) as hf:
    inception_shape = hf['inception'][:].shape[1:]
    print('Inception output shape is', inception_shape)

Inception output shape is (2, 3, 2048)


In [5]:
input_layer = layers.Input(inception_shape)
x = input_layer
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(nb_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=input_layer, outputs=x)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2, 3, 2048)        0         
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 11)                11275     
Total params: 2,109,451
Trainable params: 2,109,451
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
batch_size = 64

from keras import metrics
from functools import partial

top_3_k_categorical_accuracy = partial(metrics.top_k_categorical_accuracy, k=3)
top_3_k_categorical_accuracy.__name__ = 'top_3'

model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy', top_3_k_categorical_accuracy])

In [7]:
history = model.fit_generator(train_generator(train_dir, batch_size),
                    steps_per_epoch=train_samples_count * 50 // batch_size, 
                    validation_data=valid_generator(valid_dir, batch_size),
                    validation_steps=valid_samples_count,
                    epochs=20)

Epoch 1/20
739/739 [==============================] - 83s 112ms/step - loss: 0.5851 - acc: 0.8212 - top_3: 0.9418 - val_loss: 1.3253 - val_acc: 0.5948 - val_top_3: 0.8116 1:26 - loss - ETA: 43s - loss:  - ETA: 40s -  - ETA: 20s -  - ETA: 17s - loss: 0.6633 - acc: 0. - ETA: 16s - loss: 0.6575 - acc: 0.7982 - top - ETA: 16s  - ETA: 13s - loss: 0.6422 - acc:  - ETA: 8s - loss: 0.61 - ETA: 4s -
Epoch 2/20
739/739 [==============================] - 72s 97ms/step - loss: 0.2104 - acc: 0.9380 - top_3: 0.9909 - val_loss: 1.3611 - val_acc: 0.6086 - val_top_3: 0.8162
Epoch 3/20
739/739 [==============================] - 72s 98ms/step - loss: 0.1476 - acc: 0.9575 - top_3: 0.9949 - val_loss: 1.3478 - val_acc: 0.6184 - val_top_3: 0.8184
Epoch 4/20
739/739 [==============================] - 72s 98ms/step - loss: 0.1060 - acc: 0.9705 - top_3: 0.9972 - val_loss: 1.3589 - val_acc: 0.6280 - val_top_3: 0.8253oss: 0.1063 - acc: 0.9705 - top_3: 0.9 - ETA: 0s - loss: 0.1062 - acc: 0.9705 - top_3: 0.997 - ET